In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp utils/opensearch_utils

In [ ]:
#| export
import pandas as pd
from opensearchpy import OpenSearch
from requests.auth import HTTPBasicAuth
from IPython.display import display
import requests
import logging
from copy import deepcopy
import ast

In [ ]:
#| export
class OpensearchDataReader(object):
    def __init__(self, host:str, port:int,user:str, password:str, logger:object=None, run_config:dict=None):
        """_summary_

        Args:
            host (str): _description_
            port (int): _description_
            user (str): _description_
            password (str): _description_
            logger (object, optional): _description_. Defaults to None.
            run_config (dict, optional): _description_. Defaults to None.
        """
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.validate_parameters()
        self.client = self.get_client()
        self.run_config = run_config
        assert self.client.ping(), 'Failed to connect to opensearch client'

        self.logger = logger if logger is not None else logging.getLogger(__name__)
    
    def validate_parameters(self):
        for i in [self.user, self.password, self.host, self.port]:
            assert i is not None, f"{i} is not set"

    def get_client(self)->OpenSearch:
        """_summary_

        Returns:
            OpenSearch: _description_
        """
        return OpenSearch(
                    hosts = [{'host':self.host, 'port': self.port}],
                    http_compress = False, # enables gzip compression for request bodies
                    http_auth = (self.user, self.password),
                    use_ssl = True,
                    verify_certs = False,
                    ssl_assert_hostname = False,
                    ssl_show_warn = False,
                )
    
    def get_list_of_index(self, city:str, sep='-')->list:
        """_summary_

        Args:
            city (str): _description_
            sep (str, optional): _description_. Defaults to '-'.

        Returns:
            list: _description_
        """
        indexes = self.client.indices.get('*')
        indexes = [ x for x in indexes if city.lower() in x.lower()]
        indexes = {x: '-'.join(x.split('-')[2:-1]) for x in indexes}
        indexes = list(set(indexes.values()))
        return indexes
    
    def get_index(self, opensearch_index:str, city:str, date:pd.Timestamp)->str:
        """_summary_

        Args:
            opensearch_index (str): _description_
            city (str): _description_
            date (pd.Timestamp): _description_

        Returns:
            str: _description_
        """
        if date is not None:
            return '-'.join([f'{city.lower()}-up',opensearch_index.lower(), date.strftime('%Y%m')])
        else:
            return '-'.join([f'{city.lower()}-up',opensearch_index.lower(),'*'])
        
    def get_data(self, opensearch_index:str, query:str)->object:
        """
        get data from opensearch. scroll is used to bypass limit of 10k document
        Args:
            opensearch_index (str): _description_
            query (str): _description_

        Returns:
            object: _description_
        """

        response = self.client.search(index=opensearch_index, body=query)
        return response
       
     

In [ ]:
#| export
class GetUIIndexSchemas(object):
    """
    Get Schema information from the UI
    Args:
        object (_type_): _description_
    """
    def __init__(self, host:str, user:str, password:str, selected_events:list):
        """_summary_

        Args:
            host (str): host address
            user (str): user name
            password (str): password
            selected_events (list): list of the events to be processed
        """
        self.user = user
        self.password = password
        self.host = host
        self.selected_events = selected_events
    
    def get_schemas(self):
        response = requests.get(self.host, auth=HTTPBasicAuth(self.user, self.password))
        if response.status_code == 200:
            return response.json()
        else:
            raise IOError
    
    def restructrue_schemas(self, data:dict)->dict:
        """
        Extract only the required information from the dic
        """
        schema_dict = dict()
        for v in data['schemas']:
            d = {}
            for k1, v1 in v['properties'].items():   
                d[k1] = v1['description']
            schema_dict[v['title']] = d
        return schema_dict
    
    
    def process_schema_(self, schema_dict:dict)->dict:
        """
        Select the required events
        """
        # delete the not required 
        schema_dict = {k:v for k, v in schema_dict.items() if k in self.selected_events}
        return schema_dict
    
    def run(self):
        response = self.get_schemas()
        schema_dict = self.restructrue_schemas(response)
        schema_dict = self.process_schema_(schema_dict=schema_dict)
        return schema_dict

In [ ]:
#| export
def recursive_literal_eval(data):
    """
    Recursively applies ast.literal_eval to parse nested strings.
    """
    if isinstance(data, str):
        try:
            # Attempt to parse the string
            parsed = ast.literal_eval(data)
            # Recursively evaluate the parsed result
            return recursive_literal_eval(parsed)
        except (ValueError, SyntaxError):
            # Return original string if parsing fails
            return data
    elif isinstance(data, dict):
        # Recursively apply to dictionary values
        return {key: recursive_literal_eval(value) for key, value in data.items()}
    elif isinstance(data, list):
        # Recursively apply to list elements
        return [recursive_literal_eval(item) for item in data]
    elif isinstance(data, tuple):
        # Recursively apply to tuple elements
        return tuple(recursive_literal_eval(item) for item in data)
    else:
        # Return as-is for other types
        return data